<a href="https://colab.research.google.com/github/Robinjean/Project_Covid19_Fake_news_detection_using_RNN_LSTM_GRU_transformers/blob/main/Covid19_fake_news_detection_using_BERT_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

# Import libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
%matplotlib inline

warnings.filterwarnings('ignore')

#Load datasets

In [3]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Load dataset files
train_path = '/content/drive/My Drive/AMMI_FINAL_RESEARCH_PROJECT/Covid_fake_news/Constraint_English_Train - Sheet1.csv'
val_path = '/content/drive/My Drive/AMMI_FINAL_RESEARCH_PROJECT/Covid_fake_news/Constraint_English_Val - Sheet1.csv'
test_path = '/content/drive/My Drive/AMMI_FINAL_RESEARCH_PROJECT/Covid_fake_news/english_test_with_labels - Sheet1.csv'

# Load datasets
train_df = pd.read_csv(train_path)
val_df = pd.read_csv(val_path)
test_df = pd.read_csv(test_path)

Mounted at /content/drive


In [4]:
df = pd.concat([train_df,val_df, test_df])

In [5]:
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [6]:
# Map labels to numerical values
label_mapping = {'real': 1, 'fake': 0}
df['label'] = df['label'].map(label_mapping)

In [7]:
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,1
1,2,States reported 1121 deaths a small rise from ...,1
2,3,Politically Correct Woman (Almost) Uses Pandem...,0
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,1
4,5,Populous states can generate large case counts...,1


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10700 entries, 0 to 2139
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10700 non-null  int64 
 1   tweet   10700 non-null  object
 2   label   10700 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 334.4+ KB


In [9]:
df['label'].value_counts() #===> the data is imbalaced

label
1    5600
0    5100
Name: count, dtype: int64

#Data Preprocessing

Lowercasing

In [10]:
df['lower_text'] = df['tweet'].str.lower()
df['lower_text']

0       the cdc currently reports 99031 deaths. in gen...
1       states reported 1121 deaths a small rise from ...
2       politically correct woman (almost) uses pandem...
3       #indiafightscorona: we have 1524 #covid testin...
4       populous states can generate large case counts...
                              ...                        
2135    #coronavirusupdates: state-wise details of tot...
2136    tonight 12(midnight) onwards disaster manageme...
2137    296 new cases of #covid19nigeria; plateau-85 e...
2138    rt @cdcemergency: #dyk? @cdcgov’s one-stop sho...
2139    more than half of pregnant women recently admi...
Name: lower_text, Length: 10700, dtype: object

 Removing URLs and special characters

In [11]:
df['text'] = df['lower_text'].apply(lambda x: re.sub(r'http\S+', ' ', x))
df['text'] = df['lower_text'].apply(lambda x: re.sub(r'\S*@\S*\s?', ' ', x))
df['text'] = df['lower_text'].apply(lambda x: re.sub(r'\s+', ' ', x))
df['text'] = df['lower_text'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))

In [12]:
df['text']

0       the cdc currently reports 99031 deaths  in gen...
1       states reported 1121 deaths a small rise from ...
2       politically correct woman  almost  uses pandem...
3        indiafightscorona  we have 1524  covid testin...
4       populous states can generate large case counts...
                              ...                        
2135     coronavirusupdates  state wise details of tot...
2136    tonight 12 midnight  onwards disaster manageme...
2137    296 new cases of  covid19nigeria  plateau 85 e...
2138    rt  cdcemergency   dyk   cdcgov s one stop sho...
2139    more than half of pregnant women recently admi...
Name: text, Length: 10700, dtype: object

Removing stopwords

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
stop_words = stopwords.words('english')

In [15]:
extra = extra = ['https','t','co']

Adding the elements of extra to the stopwords

In [16]:
for c in extra:
  stop_words.append(c)

In [17]:
df['text_clean'] = df['text'].apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))

In [18]:
df['text_clean']

0       cdc currently reports 99031 deaths general dis...
1       states reported 1121 deaths small rise last tu...
2       politically correct woman almost uses pandemic...
3       indiafightscorona 1524 covid testing laborator...
4       populous states generate large case counts loo...
                              ...                        
2135    coronavirusupdates state wise details total co...
2136    tonight 12 midnight onwards disaster managemen...
2137    296 new cases covid19nigeria plateau 85 enugu ...
2138    rt cdcemergency dyk cdcgov one stop shop covid...
2139    half pregnant women recently admitted uk hospi...
Name: text_clean, Length: 10700, dtype: object

Stemming

In [19]:
from nltk.stem import PorterStemmer

# Initialize the Porter Stemmer
porter = PorterStemmer()

# Define a function for stemming
def stem_text(text):
    # Tokenize the text
    words = text.split()
    # Stem each word using Porter Stemmer
    stemmed_words = [porter.stem(word) for word in words]
    # Join the stemmed words back into a single string
    stemmed_text = " ".join(stemmed_words)
    return stemmed_text

In [20]:
# Apply stemming to the 'text_clean' column
df['text_clean'] = df['text_clean'].apply(stem_text)

# Display the preprocessed data
df['text_clean']

0       cdc current report 99031 death gener discrep d...
1       state report 1121 death small rise last tuesda...
2       polit correct woman almost use pandem excus re...
3       indiafightscorona 1524 covid test laboratori i...
4       popul state gener larg case count look new cas...
                              ...                        
2135    coronavirusupd state wise detail total confirm...
2136    tonight 12 midnight onward disast manag act im...
2137    296 new case covid19nigeria plateau 85 enugu 4...
2138    rt cdcemerg dyk cdcgov one stop shop covid19 r...
2139    half pregnant women recent admit uk hospit cov...
Name: text_clean, Length: 10700, dtype: object

#Transformer Model BERT


Mixed precision training uses both 16-bit and 32-bit floating-point numbers to speed up training and reduce memory usage

In [21]:
# Enable mixed precision training
tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [22]:
# Parameters
max_seq_length = 128

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize input sequences for both training and testing
def tokenize_and_pad(texts, max_len):
    tokenized_data = tokenizer.batch_encode_plus(
        texts,
        add_special_tokens=True,
        max_length=max_len,
        truncation=True,
        padding='max_length',
        return_tensors='tf'
    )
    return tokenized_data

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [23]:
# Tokenize and pad sequences
tokenized_train = tokenize_and_pad(df['text_clean'], max_seq_length)

In [24]:
# Convert to numpy arrays
input_ids = tokenized_train['input_ids'].numpy()
attention_masks = tokenized_train['attention_mask'].numpy()

In [25]:
# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['label'])

In [26]:
# Split data into training and testing sets
x_train, x_test, y_train, y_test, attention_masks_train, attention_masks_test = train_test_split(
    input_ids,
    labels,
    attention_masks,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

In [27]:
# Load pre-trained BERT model
model_bert = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model_bert.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [30]:
# Train the model
history_bert = model_bert.fit(
    [x_train, attention_masks_train],
    y_train,
    epochs=3,
    batch_size=64,
    validation_split=0.2
)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <gast.gast.Expr object at 0x7fce68488820>
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <gast.gast.Expr object at 0x7fce68488820>
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
107/107 [==============================] - 184s 848ms/step - loss: 0.2988 - accuracy: 0.8854 - val_loss: 0.1896 - val_accuracy: 0.9252
Epoch 2/3
107/107 [==============================] - 79s 736ms/step - loss: 0.1166 - accuracy: 0.9581 - val_loss: 0.1476 - val_accuracy: 0.9422
Epoch 3/3
107/107 [==============================] - 76s 708ms/step - loss: 0.0517 - accuracy: 0.9822 - val_loss: 0.1571 - val_accuracy: 0.9480


In [31]:
# Evaluate the model
results = model_bert.evaluate([x_test, attention_masks_test], y_test)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1] * 100:.2f}%")

67/67 [==============================] - 8s 119ms/step - loss: 0.1479 - accuracy: 0.9486
Test Loss: 0.14794909954071045, Test Accuracy: 94.86%
